In [2]:
#!pip install boto3

## Datalake transient/temperary zone: extract files for landing mechanism
# implemented by lambda or airflow

In [10]:

import boto3
s3 = boto3.client('s3')
result = s3.list_objects_v2(Bucket='video-source-data')

print(result["Contents"])

[{'Key': 'video_data_new.csv', 'LastModified': datetime.datetime(2024, 7, 5, 2, 52, 57, tzinfo=tzutc()), 'ETag': '"dbc7907b5835dd80907cb96936a7138e-8"', 'Size': 133953954, 'StorageClass': 'STANDARD'}, {'Key': 'video_data_new_id.csv', 'LastModified': datetime.datetime(2024, 7, 10, 6, 21, 30, tzinfo=tzutc()), 'ETag': '"c0f83d1c0608ea15dcc09aadd98c6795-10"', 'Size': 162265146, 'StorageClass': 'STANDARD'}, {'Key': 'video_data_new_id_2024-07-09.csv', 'LastModified': datetime.datetime(2024, 7, 10, 6, 50, 57, tzinfo=tzutc()), 'ETag': '"c0f83d1c0608ea15dcc09aadd98c6795-10"', 'Size': 162265146, 'StorageClass': 'STANDARD'}, {'Key': 'video_data_new_id_20240709.csv', 'LastModified': datetime.datetime(2024, 7, 10, 6, 50, 57, tzinfo=tzutc()), 'ETag': '"c0f83d1c0608ea15dcc09aadd98c6795-10"', 'Size': 162265146, 'StorageClass': 'STANDARD'}, {'Key': 'video_data_new_id_limit_20240710.csv', 'LastModified': datetime.datetime(2024, 7, 11, 4, 31, 7, tzinfo=tzutc()), 'ETag': '"998488a6acf6dfea39ab4deea72c7285

In [11]:
import boto3
from botocore.exceptions import NoCredentialsError
from datetime import *
import re

def list_files_with_date(bucket_name):
    s3 = boto3.client('s3')
    result = s3.list_objects_v2(Bucket=bucket_name)
    files_to_move = []

    date_patterns = [
        r'\d{4}-\d{2}-\d{2}',  # YYYY-MM-DD
        r'\d{4}\d{2}\d{2}',    # YYYYMMDD
        r'\d{2}-\d{2}-\d{4}',  # DD-MM-YYYY
        r'\d{2}/\d{2}/\d{4}',  # DD/MM/YYYY
        r'\d{4}\\\d{2}\\\d{2}', # YYYY\MM\DD
        r'\d{4}/\d{2}/\d{2}',   # YYYY/MM/DD
        # Add more patterns as needed
    ]

    yesterday = (datetime.now() - timedelta(1)).date()

    if 'Contents' in result:
        for obj in result['Contents']:
            if obj['Key'].endswith('.csv'):  # 检查是否是CSV文件
                for pattern in date_patterns:
                    match = re.search(pattern, obj['Key'])
                    if match:
                        date_str = match.group()
                        print(date_str)
                        try:
                            if '\\' in date_str:
                                date_str = date_str.replace('\\', '-')
                            file_date = datetime.strptime(date_str, '%Y-%m-%d').date()
                        except ValueError:
                            try:
                                file_date = datetime.strptime(date_str, '%Y%m%d').date()
                            except ValueError:
                                try:
                                    file_date = datetime.strptime(date_str, '%d-%m-%Y').date()
                                except ValueError:
                                    try:
                                        file_date = datetime.strptime(date_str, '%d/%m/%Y').date()
                                    except ValueError:
                                        try:
                                            file_date = datetime.strptime(date_str, '%Y/%m/%d').date()
                                        except ValueError:
                                            continue
                        if file_date == yesterday:
                            print(obj['Key'])
                            files_to_move.append(obj['Key'])
                            break
    
    return files_to_move

list_files_with_date('video-source-data')

2024-07-09
20240709
20240710
video_data_new_id_limit_20240710.csv


['video_data_new_id_limit_20240710.csv']

s3://your-raw-bucket/
s3://your-staging-bucket/
s3://your-curated-bucket/
s3://your-analytical-bucket/

In [12]:
import boto3
from botocore.exceptions import NoCredentialsError
from datetime import *

def move_s3_object(src_bucket_name, src_object_name, dest_bucket_name, dest_object_name):
    s3 = boto3.resource('s3')
    try:
        # Copy the object to the new location
        copy_source = {
            'Bucket': src_bucket_name,
            'Key': src_object_name
        }
        s3.meta.client.copy(copy_source, dest_bucket_name, dest_object_name)
        
        print(f"File {src_object_name} moved from {src_bucket_name} to {dest_bucket_name}/{dest_object_name}")
    except NoCredentialsError:
        print("Credentials not available")

# Replace these values with your bucket names
src_bucket_name = 'video-source-data'
dest_bucket_name = 'video-transient-bucket'

folder_name = datetime.now().strftime('%Y-%m-%d')  # 目标文件夹路径

files_to_move = list_files_with_date(src_bucket_name)

for file_key in files_to_move:
    dest_key = f'{folder_name}/{file_key.split("/")[-1]}'  # 将文件移到目标文件夹
    move_s3_object(src_bucket_name, file_key, dest_bucket_name, dest_key)


2024-07-09
20240709
20240710
video_data_new_id_limit_20240710.csv
File video_data_new_id_limit_20240710.csv moved from video-source-data to video-transient-bucket/2024-07-11/video_data_new_id_limit_20240710.csv
